In [1]:
from utils import *
from data import *
from model import STraTS

import warnings
import time
import logging
logger = logging.getLogger(__name__)
from datetime import datetime
import os
from tqdm import tqdm
import torch
import torch.nn
from torch.utils.data import DataLoader

In [2]:
data_dir = './mortality_datasets'
fp16 = True
cpu = True
train_batch_size, eval_batch_size = 32, 32
d,N, he, dropout = 50, 2, 4, 0.2
learning_rate = 0.0005
forecast_model_weights = './logs/20231019_2121/best_forecasting_model.pth'

In [3]:
train_dataset, val_dataset, test_dataset, V, D = load_mortality_dataset(data_dir)

In [4]:
model = STraTS(
    # No. of Demographics features
    D=D,
    # No. of Variable Embedding Size
    V=V,
    d=d,
    N=N,
    he=he,
    dropout=dropout,
    forecast=False
)

varis_stack: 6500
values_stack: 364
times_stack: 364
cont_stack: 39508
attn_stack: 5200
demo_stack: 5350
output_stack: 101
Total Parameters: 57387


In [5]:
model

STraTS(
  (varis_stack): Embedding(130, 50)
  (values_stack): CVE(
    (stack): Sequential(
      (0): Linear(in_features=1, out_features=7, bias=True)
      (1): Tanh()
      (2): Linear(in_features=7, out_features=50, bias=False)
    )
  )
  (times_stack): CVE(
    (stack): Sequential(
      (0): Linear(in_features=1, out_features=7, bias=True)
      (1): Tanh()
      (2): Linear(in_features=7, out_features=50, bias=False)
    )
  )
  (cont_stack): Transformer(
    (dropout_layer): Dropout(p=0.2, inplace=False)
    (identity): Identity()
  )
  (attn_stack): Attention(
    (stack): Sequential(
      (0): Linear(in_features=50, out_features=100, bias=True)
      (1): Tanh()
      (2): Linear(in_features=100, out_features=1, bias=False)
    )
    (softmax): Softmax(dim=-2)
  )
  (demo_stack): Sequential(
    (0): Linear(in_features=2, out_features=100, bias=True)
    (1): Tanh()
    (2): Linear(in_features=100, out_features=50, bias=True)
    (3): Tanh()
  )
  (output_stack): Sequential

In [6]:
forecast_weights = torch.load(forecast_model_weights, map_location=torch.device('cpu'))

In [7]:
for k in list(forecast_weights.keys()):
    if k.startswith('output_stack'):
        del forecast_weights[k]

In [8]:
model_dict = model.state_dict()
model_dict.update(forecast_weights)
model.load_state_dict(model_dict)

<All keys matched successfully>

In [9]:
test_dataloader = DataLoader(test_dataset, batch_size=5)

In [10]:
batch = next(iter(test_dataloader))
X_demos, X_times, X_values, X_varis, Y = batch

In [11]:
Y_pred = model(X_demos, X_times, X_values, X_varis)

In [12]:
Y_pred

tensor([0.5275, 0.6058, 0.5104, 0.4966, 0.4790], grad_fn=<ViewBackward>)

In [13]:
Y

tensor([1., 0., 0., 0., 0.])

In [14]:
mortality_loss(Y, Y_pred)

tensor(0.7246, grad_fn=<BinaryCrossEntropyBackward>)

In [ ]:
m = nn.Sigmoid()
loss = nn.BCELoss()
input = torch.randn(3, requires_grad=True)
target = torch.empty(3).random_(2)
output = loss(m(input), target)